### Postprocess the instrumental/vocal label file

In [1]:
import numpy as np
import pandas as pd
from os.path import join

In [2]:
# DEFINE PATHS and FILE PATTERNS

METADATA_PATH = 'metadata_new'

# INPUT
# here, %s will be replaced by 'instrumental'
LABEL_FILE_PATTERN = join(METADATA_PATH, 'ismir2018_tut_part_2_instrumental_labels_orig.csv') 

# OUTPUT
OUT_LABEL_FILE_PATTERN = join(METADATA_PATH, 'ismir2018_tut_part_2_instrumental_labels.csv') 

In [3]:
labels = pd.read_csv(LABEL_FILE_PATTERN, index_col=0) #, sep='\t')

labels.head()

,instrumental,vocal
14,0.0,1.0
17,0.0,1.0
88,1.0,0.0
103,0.0,1.0
128,0.0,1.0


In [4]:
# check class distribution
labels.sum()

instrumental     443.0
vocal           1283.0
dtype: float64

In [5]:
# we already removed the tracks that were *neither* instrumental *nor* vocal
labels.sum(axis=1).min()

1.0

In [6]:
# unfortunately a few songs are labeled *both* instrumental *and* vocal
labels.sum(axis=1).max()

2.0

<b>We remove the ambiguously annotated tracks by using XOR to keep only tracks that are *either* instrumental *or* vocal</b><br>

In [7]:
retain = np.logical_xor(labels['instrumental'], labels['vocal'])
retain.head()

14     True
17     True
88     True
103    True
128    True
Name: instrumental, dtype: bool

In [8]:
# keep only ones that are set "True" in retain
n_orig = len(labels)
n_retain = sum(retain)

labels = labels[retain]

print("For instrumental vs. vocal, from originally", n_orig, "input examples, we can only retain",n_retain, "trusted ones in our groundtruth")

For instrumental vs. vocal, from originally 1703 input examples, we can only retain 1680 trusted ones in our groundtruth


In [9]:
# check class distribution after removal of ambigous tracks
labels.sum()

instrumental     420.0
vocal           1260.0
dtype: float64

In [10]:
labels.head()

,instrumental,vocal
14,0.0,1.0
17,0.0,1.0
88,1.0,0.0
103,0.0,1.0
128,0.0,1.0


In [11]:
# keep only 1 column as now they are redundant (one is the inverse of the other)
labels = labels[['instrumental']]
labels.head()

,instrumental
14,0.0
17,0.0
88,1.0
103,0.0
128,0.0


In [12]:
# double-check number of instrumental tracks
labels.sum()

instrumental    420.0
dtype: float64

In [13]:
# export file under new filename
labels.to_csv(OUT_LABEL_FILE_PATTERN)
print("Wrote " + OUT_LABEL_FILE_PATTERN)

Wrote metadata_new/ismir2018_tut_part_2_instrumental_labels.csv
